<a href="https://githubtocolab.com/giswqs/geemap/blob/master/examples/notebooks/31_unsupervised_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

Uncomment the following line to install [geemap](https://geemap.org) if needed.

In [ ]:
# !pip install geemap

# Machine Learning with Earth Engine - Unsupervised Classification

## Unsupervised classification algorithms available in Earth Engine

Source: https://developers.google.com/earth-engine/clustering

The `ee.Clusterer` package handles unsupervised classification (or clustering) in Earth Engine. These algorithms are currently based on the algorithms with the same name in [Weka](http://www.cs.waikato.ac.nz/ml/weka/). More details about each Clusterer are available in the reference docs in the Code Editor.

Clusterers are used in the same manner as classifiers in Earth Engine. The general workflow for clustering is:

1. Assemble features with numeric properties in which to find clusters.
2. Instantiate a clusterer. Set its parameters if necessary.
3. Train the clusterer using the training data.
4. Apply the clusterer to an image or feature collection.
5. Label the clusters.

The training data is a `FeatureCollection` with properties that will be input to the clusterer. Unlike classifiers, there is no input class value for an `Clusterer`. Like classifiers, the data for the train and apply steps are expected to have the same number of values. When a trained clusterer is applied to an image or table, it assigns an integer cluster ID to each pixel or feature.

Here is a simple example of building and using an ee.Clusterer:

![](https://i.imgur.com/IcBapEx.png)

## Step-by-step tutorial

### Import libraries

In [1]:
import ee
import geemap

### Create an interactive map

In [2]:
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

### Add data to the map

In [6]:
# point = ee.Geometry.Point([-99.4439, 37.7538])
point = ee.Geometry.Point([-89.7719, 19.8799])

image = (
    ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
    .filterBounds(point)
    .filterDate('2019-01-01', '2019-12-31')
    .sort('CLOUD_COVER')
    .first()
    .select('B[1-7]')
)

vis_params = {'min': 0, 'max': 3000, 'bands': ['B5', 'B4', 'B3']}

Map.centerObject(point, 8)
Map.addLayer(image, vis_params, "Landsat-8")

### Check image properties

In [7]:
props = geemap.image_props(image)
props.getInfo()

{'CLOUD_COVER': 0.02,
 'CLOUD_COVER_LAND': 0.02,
 'EARTH_SUN_DISTANCE': 0.992617,
 'ESPA_VERSION': '2_23_0_1b',
 'GEOMETRIC_RMSE_MODEL': 7.902,
 'GEOMETRIC_RMSE_MODEL_X': 5.856,
 'GEOMETRIC_RMSE_MODEL_Y': 5.305,
 'IMAGE_DATE': '2019-03-08',
 'IMAGE_QUALITY_OLI': 9,
 'IMAGE_QUALITY_TIRS': 9,
 'LANDSAT_ID': 'LC08_L1TP_020046_20190308_20190324_01_T1',
 'LEVEL1_PRODUCTION_DATE': 1553471444000,
 'NOMINAL_SCALE': 30,
 'PIXEL_QA_VERSION': 'generate_pixel_qa_1.6.0',
 'SATELLITE': 'LANDSAT_8',
 'SENSING_TIME': '2019-03-08T16:22:10.0501829Z',
 'SOLAR_AZIMUTH_ANGLE': 130.871262,
 'SOLAR_ZENITH_ANGLE': 36.237225,
 'SR_APP_VERSION': 'LaSRC_1.3.0',
 'WRS_PATH': 20,
 'WRS_ROW': 46,
 'system:asset_size': '558.760301 MB',
 'system:band_names': ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7'],
 'system:id': 'LANDSAT/LC08/C01/T1_SR/LC08_020046_20190308',
 'system:index': 'LC08_020046_20190308',
 'system:time_end': '2019-03-08 16:22:10',
 'system:time_start': '2019-03-08 16:22:10',
 'system:version': 1563396459

In [8]:
props.get('IMAGE_DATE').getInfo()

'2019-03-08'

In [9]:
props.get('CLOUD_COVER').getInfo()

0.02

### Make training dataset

There are several ways you can create a region for generating the training dataset.

- Draw a shape (e.g., rectangle) on the map and the use `region = Map.user_roi`
- Define a geometry, such as `region = ee.Geometry.Rectangle([-122.6003, 37.4831, -121.8036, 37.8288])`
- Create a buffer zone around a point, such as `region = ee.Geometry.Point([-122.4439, 37.7538]).buffer(10000)`
- If you don't define a region, it will use the image footprint by default

In [10]:
# region = Map.user_roi
# region = ee.Geometry.Rectangle([-122.6003, 37.4831, -121.8036, 37.8288])
region = ee.Geometry.Point([-89.7719, 19.8799]).buffer(10000)

In [11]:
# Make the training dataset.
training = image.sample(
    **{
        'region': region,
        'scale': 30,
        'numPixels': 5000,
        'seed': 0,
        'geometries': True,  # Set this to False to ignore geometries
    }
)

Map.addLayer(training, {}, 'training', False)
Map

Map(bottom=4109.0, center=[12.511665400971031, -73.58642578125001], controls=(WidgetControl(options=['position…

### Train the clusterer

In [12]:
# Instantiate the clusterer and train it.
n_clusters = 5
clusterer = ee.Clusterer.wekaKMeans(n_clusters).train(training)

### Classify the image

In [13]:
# Cluster the input using the trained clusterer.
result = image.cluster(clusterer)

# # Display the clusters with random colors.
Map.addLayer(result.randomVisualizer(), {}, 'clusters')
Map

Map(bottom=232882.0, center=[19.886848881896228, -89.77306365966798], controls=(WidgetControl(options=['positi…

### Label the clusters

In [15]:
legend_keys = ['One', 'Two', 'Three', 'Four', 'ect']
legend_colors = ['#8DD3C7', '#FFFFB3', '#BEBADA', '#FB8072', '#80B1D3']

# Reclassify the map
result = result.remap([0, 1, 2, 3, 4], [1, 2, 3, 4, 5])

Map.addLayer(
    result, {'min': 1, 'max': 5, 'palette': legend_colors}, 'Labelled clusters'
)
Map.add_legend(
    legend_keys=legend_keys, legend_colors=legend_colors, position='bottomright'
)
Map

Map(bottom=1847579.0, center=[20.962802166960653, -89.74976062774658], controls=(WidgetControl(options=['posit…

### Visualize the result

In [16]:
print('Change layer opacity:')
cluster_layer = Map.layers[-1]
cluster_layer.interact(opacity=(0, 1, 0.1))

Change layer opacity:


Box(children=(FloatSlider(value=1.0, description='opacity', max=1.0),))

### Export the result

Export the result directly to your computer:

In [19]:
import os

out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
out_file = os.path.join(out_dir, 'cluster.tif')

In [22]:
geemap.ee_export_image(result, filename=out_file, scale=60)

Generating URL ...
Please wait ...
Data downloaded to C:\Users\PC\Downloads\cluster.tif


Export the result to Google Drive:

In [ ]:
geemap.ee_export_image_to_drive(
    result, description='clusters', folder='export', scale=90
)